In [10]:
import pandas as pd
import numpy as np
import mysql.connector
import pickle
import math
import operator
import random
from sqlalchemy import create_engine

In [11]:
class distancias:

    def __init__(self):
        
        pass

    def distancia_euclidiana(self, vector1, vector2):

        self.vectorA, self.vectorB = vector1, vector2

        if len(self.vectorA) != len(self.vectorB):
            raise ValueError("Vectores deben ser de la misma longitud")
        distance = 0.0

        for i in range(len(self.vectorA) - 1):
            distance += (self.vectorA[i] - self.vectorB[i])**2
        return (distance)**0.5
    
    def distancia_manhattan(self, vector1, vector2):

        self.vectorA, self.vectorB = vector1, vector2

        if len(self.vectorA) != len(self.vectorB):
            raise ValueError("Vectores deben ser de la misma longitud")
        return np.abs(np.array(self.vectorA) - np.array(self.vectorB)).sum()
    
    def distancia_hamming(self, vector1, vector2):

        self.vectorA, self.vectorB = vector1, vector2

        if len(self.vectorA) != len(self.vectorB):
            raise ValueError("Vectores deben ser de la misma longitud")
        return sum(el1 != el2 for el1, el2 in zip(self.vectorA, self.vectorB))

In [12]:
class KNN:

    def __init__(self, k = 3, distanciaM = 'euclidiana'):

        pass

    def fit(self, xTrain, yTrain):

        assert len(xTrain) == len(yTrain)
        self.trainData = xTrain
        self.trainLabels = yTrain

    def getNeighbors(self, testRow):

        cDistancia = distancias()
        distancia = []

        for i, trainRow in enumerate(self.trainData):
            if self.distanciaM == 'euclidiana':
                distancia.append([trainRow, cDistancia.distancia_euclidiana(testRow, trainRow), self.trainLabels[i]])
            elif self.distanciaM == 'manhattan':
                distancia.append([trainRow, cDistancia.distancia_euclidiana(testRow, trainRow), self.trainLabels[i]])
            elif self.distanciaM == 'hamming':
                distancia.append([trainRow, cDistancia.distancia_euclidiana(testRow, trainRow), self.trainLabels[i]])
            distancia.sort(key=operator.itemgetter(1))

        neighbors = []

        for index in range(self.k):
            neighbors.append(distancia[index])
        return neighbors
    
    def prediccion(self, xTest, k, distanciaM):

        self.testData = xTest
        self.k = k
        self.distanciaM = distanciaM
        predicciones = []

        for i, testCase in enumerate(self.testData):
            neighbors = self.getNeighbors(testCase)
            salida = [row[-1] for row in neighbors]
            prediccion = max(set(salida), key=salida.count)
            predicciones.append(prediccion)

        return predicciones


In [15]:
def printMetricas(actual, predictions):
    assert len(actual) == len(predictions)
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predictions[i]:
            correct += 1
    return (correct / float(len(actual)) * 100.0)

In [16]:
class kFold:

    def __init__(self):

        pass

    def Validacion_Cross(self, dataset, nFolds):

        dataSplit = list()
        dataC = list(dataset)
        tFold = int(len(dataset) / nFolds)

        for _ in range(nFolds):
            fold = list()
            while len(fold) < tFold:
                index = randrange(len(dataC))
                fold.append(dataC.pop(index))
            dataSplit.append(fold)
        return dataSplit
    
    def kfc_evaluacion(self, dataset, nFolds, *args):
        knn = KNN()
        folds = self.Validacion_Cross(dataset, nFolds)
        puntuaciones = list()

        for fold in folds:
            trainSet = list(folds)
            trainSet.remove(fold)
            trainSet = sum(trainSet, [])
            testSet = list()

            for row in fold:
                rowC = list(row)
                testSet.append(rowC)
            
            trainLabels = [row[-1] for row in trainSet]
            trainSet = [train[:-1] for train in trainSet]
            knn.fit(trainSet, trainLabels)

            actual = [row[-1] for row in trainSet]
            testSet = [test[:-1] for test in testSet]

            prediccion = knn.prediccion(testSet, *args)

            accuracy = printMetricas(actual, prediccion)
            puntuaciones.append(accuracy)

            print('*'*20)
            print('Scores: %s' % puntuaciones)
            print('*'*20)        
            print('\nMaximum Accuracy: %3f%%' % max(puntuaciones))
            print('\nMean Accuracy: %.3f%%' % (sum(puntuaciones)/float(len(puntuaciones))))

